***SIMULACIONES***

Simulación Baja Resolución

In [25]:
import pandas as pd
import matplotlib.pyplot as plt
from gurobipy import Model, GRB, quicksum
import networkx as nx
import warnings 
warnings.filterwarnings('ignore')
import numpy as np
import random
import gurobipy as gp

In [61]:
# LOAD DATA
#Compatibility
compatibility = pd.read_csv('/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/compatibilidad_total.csv', index_col=0)
compatibility.index = range(len(compatibility))
compatibility.columns = range(len(compatibility.columns))
#Pairs, recipients and donors
pairs = pd.read_csv('/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/parejas.csv',index_col=0)
recipients = pd.read_csv('/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/receptores.csv',index_col=0)
donors = pd.read_csv('/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/donantes.csv',index_col=0)
#Weights
hla_hr = pd.read_csv("/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/peso_ar.csv",index_col=0)
hla_lr = pd.read_csv("/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/peso_BR.csv",index_col=0)
hla_eplet = pd.read_csv("/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/peso_eplet.csv",index_col=0)
hla_hr.columns = hla_hr.columns.astype(int)
hla_lr.columns = hla_lr.columns.astype(int)
hla_eplet.columns = hla_eplet.columns.astype(int)

In [62]:
# Data of each locus 
base = '/Users/valentina/Library/CloudStorage/OneDrive - UAI/TESIS/CODIGO/Datas_simulaciones/locis_por_separado/BR/'
mats = {
    'A': 'mismatch_BR_A.csv',
    'B': 'mismatch_BR_B.csv',
    'C': 'mismatch_BR_C.csv',
    'DQ': 'mismatch_BR_DQ.csv',
    'DR': 'mismatch_BR_DR.csv'
}

matrix_mismatch = {}
for locus, archive in mats.items():
    matrix_mismatch[locus] = pd.read_csv(base + archive, index_col=0)

mismatch_ClassI = (
    matrix_mismatch['A'] +
    matrix_mismatch['B'] +
    matrix_mismatch['C']
)

mismatch_DQ = matrix_mismatch['DQ']
mismatch_DR = matrix_mismatch['DR']

In [63]:
# Mismatch to HLA score
HLA_DR = 2 - mismatch_DR
HLA_DQ = 2- mismatch_DQ
HLA_C1 = 6 - mismatch_ClassI

In [64]:
# Creation of initial graph with graph resolution and a minimum of quality (k)
def create_graph(pairs, compatibility, hla1, k):
    G = nx.DiGraph()
    added_edges = 0
    for i in pairs.index:
        for j in pairs.index:
            if compatibility.at[i, j] == 1 and hla1.at[i, j] >= k:
                G.add_edge(j, i, weight=hla1.at[i, j])
                added_edges += 1
    return G

In [65]:
# Changing initial weights to optimization weights on the initial graph arcs
def changing_resolution_weights(G, hla2):
    for u, v, data in G.edges(data=True):
        try:
            data['weight'] = hla2.iloc[int(v), int(u)]
        except KeyError:
            print(f"No se encontró peso para el arco ({u}, {v})")
        except Exception as e:
            print(f"Error al actualizar peso para el arco ({u}, {v}): {str(e)}")


In [66]:
# FOR OPTUNA

def optimization(G, multipliers_ethnicity, l=3, k=3):
    total_cycles = list(nx.simple_cycles(G, length_bound=3))
    valid_cycles = [cycle for cycle in total_cycles if len(cycle) <= l and all(G[u][v]['weight'] >= k for u, v in zip(cycle, cycle[1:] + cycle[:1]))]
    
    P = len(G.nodes())  
    Z = 10



    # # ---- imprimir desglose ----
    # for cycle in valid_cycles[:5]:  
    #     print(f"\nCiclo {cycle}:")
    #     term = 0
    #     for u, v in zip(cycle, cycle[1:] + cycle[:1]):
    #         eth = recipients.loc[v, 'ETHCAT']
    #         mult = multipliers_ethnicity.get(eth, 1.0)
    #         w = G[u][v]['weight']

    #         part1 = mult * 1
    #         part2 = mult * (w / (P * Z))

    #         term = term + part1 + part2


    #         print(f"  {u}->{v} | ETHCAT={eth} | multiplier={mult}")
    #         print(f"    multiplier * 1 = {part1:.6f}")
    #         print(f"    + multiplier * (w/(P*Z)) = {part2:.6f}  (w={w}, P={P}, Z={Z})")

    #     print("Term:", term )
    #     print("-" * 50)


    m = Model("optimization")
    m.setParam('OutputFlag', 0)

    x = {
        tuple(cycle): m.addVar(vtype=GRB.BINARY, 
                               name=f"x_{'_'.join(map(str, cycle))}") 
        for cycle in valid_cycles
    }

    m.setObjective(
        quicksum(
            x[tuple(cycle)] * 
            #(
                sum(
                multipliers_ethnicity.get(recipients.loc[v, 'ETHCAT'], 1.0) *
                (1 + (G[u][v]['weight'] / (P * Z)))
                for u, v in zip(cycle, cycle[1:] + cycle[:1])
            )
            #)/P
            
            for cycle in valid_cycles
        ),
        GRB.MAXIMIZE
    )

    for i in G.nodes():
        m.addConstr(quicksum(x[tuple(cycle)] for cycle in valid_cycles if i in cycle) <= 1, 
                    name=f"node_usage_{i}")

    try:
        m.optimize()
    except gp.GurobiError as e:
        print(f"⚠️ Error of Gurobi: {e}")
      
        return None, [], True  

    G_optimal = nx.DiGraph()
    selected_cycles = []

    if m.status == GRB.OPTIMAL:
        for cycle in valid_cycles:
            if x[tuple(cycle)].X > 0.5:
                selected_cycles.append(cycle)
                for i in range(len(cycle)):
                    u, v = cycle[i], cycle[(i + 1) % len(cycle)]
                    G_optimal.add_edge(u, v, weight=G[u][v]['weight'])


    return G_optimal, selected_cycles, False


In [ ]:
# Complete definition of the simulation

def update_matrices(indexes, df):
    return df.iloc[indexes, indexes]
frequiency_waiting_nodes = {}

def run_simulation(total_time, arrival_rate, departure_rate, match_run, pairs, compatibility, hla_lr, hla_hr, hla_eplet, multipliers_ethnicity):
    # Evaluation at different resolutions
    #np.random.seed(40)
    lr_quality_ethcat1 = []
    lr_quality_ethcat2 = []
    lr_quality_ethcat4 = []
    lr_quality_ethcat5 = []
    lr_quality_ethcat6 = []
    lr_quality_ethcat7 = []
    
    hr_quality_ethcat1 = []
    hr_quality_ethcat2 = []
    hr_quality_ethcat4 = []
    hr_quality_ethcat5 = []
    hr_quality_ethcat6 = []
    hr_quality_ethcat7 = []

    e_quality_ethcat1 = []
    e_quality_ethcat2 = []
    e_quality_ethcat4 = []
    e_quality_ethcat5 = []
    e_quality_ethcat6 = []
    e_quality_ethcat7 = []

    waiting_list = []
    waiting_times = {}  
    arrivals_by_ethcat = {}
    historial_cycles = []
    historial_departures = []
    cont=0
    available_indexes = set(pairs.index)

    
    departures_by_ethcat = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}
    HLA_classI_ethcat = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}
    HLA_DR_ethcat = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}
    HLA_DQ_ethcat = {1: [], 2: [], 4: [], 5: [], 6: [], 7: []}

    # Simulation per month
    for month in range(total_time):
     
        selected_cycles = []
       
        new_entries = np.random.poisson(arrival_rate)
        new_entries = min(new_entries, len(available_indexes))
        new_index = np.random.choice(list(available_indexes), size=new_entries, replace=False)
        cont+= len(new_index)
        waiting_list.extend(new_index)
        available_indexes.difference_update(new_index)

        for idx in new_index:
            waiting_times[idx] = {'arrival': month}
            ethnicity = recipients.loc[recipients['Nodo'] == idx, 'ETHCAT'].iloc[0]
            if ethnicity in arrivals_by_ethcat:
                arrivals_by_ethcat[ethnicity] += 1
            else:
                arrivals_by_ethcat[ethnicity] = 1
        
 
        departure = np.random.poisson(departure_rate)
        departure_indexes = [] 
        if departure:
            departure = min(departure, len(waiting_list))
            departure_indexes = np.random.choice(waiting_list, size=departure, replace=False)
            waiting_list = [idx for idx in waiting_list if idx not in departure_indexes]
            historial_departures.extend(departure_indexes)

        for idx in departure_indexes:
            ethnicity = recipients.loc[idx, 'ETHCAT']
            departures_by_ethcat[ethnicity].append(idx)

        if (month + 1) % match_run == 0:
            waiting_list_index = waiting_list.copy()
            df_waiting_list = pairs.loc[waiting_list_index]
            
            filtered_compatibility = update_matrices(waiting_list_index, compatibility)
            filtered_weight = update_matrices(waiting_list_index, hla_lr) # Insert the resolution data that you want for create the graph
            
    
            G = create_graph(df_waiting_list, filtered_compatibility, filtered_weight, k= 3) # Insert the minimum weight for create the graph 
            changing_resolution_weights(G, hla_lr) # Insert the resolution data that you want for the optimization

            
            
            G_optimal, selected_cycles,  error_gurobi = optimization(G, multipliers_ethnicity, l=3, k=3 ) # Insert the minimum quality that you want for the cycles 

            if error_gurobi:
                # Error real de Gurobi → penalizamos
                return float("inf"), None

            for u, v, data in G_optimal.edges(data=True):
                node_ethcat = recipients.loc[v, 'ETHCAT'] 
                
                # HLA for differents loci (optimization resolution)
                value_HLA_classI = HLA_C1.iloc[v, u]
                value_HLA_DR = HLA_DR.iloc[v, u]
                value_HLA_DQ = HLA_DQ.iloc[v, u]

                HLA_classI_ethcat[node_ethcat].append(value_HLA_classI)
                HLA_DR_ethcat[node_ethcat].append(value_HLA_DR)
                HLA_DQ_ethcat[node_ethcat].append(value_HLA_DQ)


                if node_ethcat == 1:
                    lr_quality_ethcat1.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat1.append(hla_hr.iloc[v, u])
                    e_quality_ethcat1.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 2:
                    lr_quality_ethcat2.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat2.append(hla_hr.iloc[v, u])
                    e_quality_ethcat2.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 4:
                    lr_quality_ethcat4.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat4.append(hla_hr.iloc[v, u])
                    e_quality_ethcat4.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 5:
                    lr_quality_ethcat5.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat5.append(hla_hr.iloc[v, u])
                    e_quality_ethcat5.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 6:
                    lr_quality_ethcat6.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat6.append(hla_hr.iloc[v, u])
                    e_quality_ethcat6.append(hla_eplet.iloc[v, u])
                elif node_ethcat == 7:
                    lr_quality_ethcat7.append(hla_lr.iloc[v, u])
                    hr_quality_ethcat7.append(hla_hr.iloc[v, u])
                    e_quality_ethcat7.append(hla_eplet.iloc[v, u])
            historial_cycles.extend(selected_cycles)

            nodes_in_cycles = [node for cycle in selected_cycles for node in cycle]
            waiting_list = [idx for idx in waiting_list if idx not in nodes_in_cycles]
            for idx in nodes_in_cycles:
                waiting_times[idx]['departure'] = month

    nodes_in_historial = [node for cycle in historial_cycles for node in cycle]

    data = []

    for idx in nodes_in_historial:
        if idx in waiting_times:
            waiting_time = waiting_times[idx]['departure'] - waiting_times[idx]['arrival']
            ethnicity = recipients.loc[idx, 'ETHCAT'] 
            data.append({'node': idx, 'waiting time (months)': waiting_time, 'ethnicity': ethnicity})

    

    df_ethnicity = pd.DataFrame(data)
    df_ethnicity['ethnicity'] = df_ethnicity['ethnicity'].astype(int)
    df_ethnicity1 = df_ethnicity[df_ethnicity['ethnicity'] == 1]
    df_ethnicity2 = df_ethnicity[df_ethnicity['ethnicity'] == 2]
    df_ethnicity4 = df_ethnicity[df_ethnicity['ethnicity'] == 4]
    df_ethnicity5 = df_ethnicity[df_ethnicity['ethnicity'] == 5]
    df_ethnicity6 = df_ethnicity[df_ethnicity['ethnicity'] == 6]
    df_ethnicity7 = df_ethnicity[df_ethnicity['ethnicity'] == 7]
    waiting_time_mean_1 = df_ethnicity1['waiting time (months)'].mean()
    waiting_time_mean_2 = df_ethnicity2['waiting time (months)'].mean()
    waiting_time_mean_4 = df_ethnicity4['waiting time (months)'].mean()
    waiting_time_mean_5 = df_ethnicity5['waiting time (months)'].mean()
    waiting_time_mean_6 = df_ethnicity6['waiting time (months)'].mean()
    waiting_time_mean_7 = df_ethnicity7['waiting time (months)'].mean()

    
    outgoing_per_ethcat = {e: len(departures_by_ethcat[e]) for e in [1,2,4,5,6,7]}
    L_per_ethcat  = {e: outgoing_per_ethcat [e] / arrivals_by_ethcat.get(e, 1) for e in outgoing_per_ethcat } 
    F_per_ethcat  = {e: len([c for c in historial_cycles for n in c if recipients.loc[n,'ETHCAT']==e]) / arrivals_by_ethcat.get(e, 1)
                   for e in [1,2,4,5,6,7]}  
    
     
    total_trasplants = sum(len(cycle) for cycle in historial_cycles)
    total_entries = sum(arrivals_by_ethcat.values())
    F_total = total_trasplants / total_entries if total_entries > 0 else 0


     

    return {
        'historial_cycles': historial_cycles,
        'historial_departures': historial_departures,
        'entries_nodes': cont,
        'arrivals_by_ethcat': arrivals_by_ethcat,
        'waiting_time_mean1':   waiting_time_mean_1 ,
        'waiting_time_mean2':   waiting_time_mean_2,
        'waiting_time_mean4':   waiting_time_mean_4,
        'waiting_time_mean5':   waiting_time_mean_5,
        'waiting_time_mean6':   waiting_time_mean_6,
        'waiting_time_mean7':   waiting_time_mean_7,
        'lr_quality_ethcat1': np.mean(lr_quality_ethcat1),
        'lr_quality_ethcat2': np.mean(lr_quality_ethcat2),
        'lr_quality_ethcat4': np.mean(lr_quality_ethcat4),
        'lr_quality_ethcat5': np.mean(lr_quality_ethcat5),
        'lr_quality_ethcat6': np.mean(lr_quality_ethcat6),
        'lr_quality_ethcat7': np.mean(lr_quality_ethcat7),

        'hr_quality_ethcat1': np.mean(hr_quality_ethcat1),
        'hr_quality_ethcat2': np.mean(hr_quality_ethcat2),
        'hr_quality_ethcat4': np.mean(hr_quality_ethcat4),
        'hr_quality_ethcat5': np.mean(hr_quality_ethcat5),
        'hr_quality_ethcat6': np.mean(hr_quality_ethcat6),
        'hr_quality_ethcat7': np.mean(hr_quality_ethcat7),

        'e_quality_ethcat1': np.mean(e_quality_ethcat1),
        'e_quality_ethcat2': np.mean(e_quality_ethcat2),
        'e_quality_ethcat4': np.mean(e_quality_ethcat4),
        'e_quality_ethcat5': np.mean(e_quality_ethcat5),
        'e_quality_ethcat6': np.mean(e_quality_ethcat6),
        'e_quality_ethcat7': np.mean(e_quality_ethcat7),
        'avg_ClassI_HLA_ethcat1': np.mean(HLA_classI_ethcat[1]), 
        'avg_ClassI_HLA_ethcat2': np.mean(HLA_classI_ethcat[2]),
        'avg_ClassI_HLA_ethcat4': np.mean(HLA_classI_ethcat[4]),
        'avg_ClassI_HLA_ethcat5': np.mean(HLA_classI_ethcat[5]),
        'avg_ClassI_HLA_ethcat6': np.mean(HLA_classI_ethcat[6]),
        'avg_ClassI_HLA_ethcat7': np.mean(HLA_classI_ethcat[7]), 
        'HLA_ClassI_total': np.mean(
                                    HLA_classI_ethcat[1] + 
                                    HLA_classI_ethcat[2] + 
                                    HLA_classI_ethcat[4] + 
                                    HLA_classI_ethcat[5] + 
                                    HLA_classI_ethcat[6] + 
                                    HLA_classI_ethcat[7]
                                ),

        'avg_DR_HLA_ethcat1': np.mean(HLA_DR_ethcat[1]), 
        'avg_DR_HLA_ethcat2': np.mean(HLA_DR_ethcat[2]),
        'avg_DR_HLA_ethcat4': np.mean(HLA_DR_ethcat[4]),
        'avg_DR_HLA_ethcat5': np.mean(HLA_DR_ethcat[5]),
        'avg_DR_HLA_ethcat6': np.mean(HLA_DR_ethcat[6]),
        'avg_DR_HLA_ethcat7': np.mean(HLA_DR_ethcat[7]), 
        'HLA_DR_total': np.mean(
                                    HLA_DR_ethcat[1] + 
                                    HLA_DR_ethcat[2] + 
                                    HLA_DR_ethcat[4] + 
                                    HLA_DR_ethcat[5] + 
                                    HLA_DR_ethcat[6] + 
                                    HLA_DR_ethcat[7]
                                ),

        'avg_DQ_HLA_ethcat1': np.mean(HLA_DQ_ethcat[1]), 
        'avg_DQ_HLA_ethcat2': np.mean(HLA_DQ_ethcat[2]),
        'avg_DQ_HLA_ethcat4': np.mean(HLA_DQ_ethcat[4]),
        'avg_DQ_HLA_ethcat5': np.mean(HLA_DQ_ethcat[5]),
        'avg_DQ_HLA_ethcat6': np.mean(HLA_DQ_ethcat[6]),
        'avg_DQ_HLA_ethcat7': np.mean(HLA_DQ_ethcat[7]), 
        'HLA_DQ_total': np.mean(
                                    HLA_DQ_ethcat[1] + 
                                    HLA_DQ_ethcat[2] + 
                                    HLA_DQ_ethcat[4] + 
                                    HLA_DQ_ethcat[5] + 
                                    HLA_DQ_ethcat[6] + 
                                    HLA_DQ_ethcat[7]
                                ),

        'F_per_ethcat': F_per_ethcat,        
        'L_per_ethcat' : L_per_ethcat,          
        'F_total': F_total,
        'lr_quality_total': np.mean(
                                    lr_quality_ethcat1 + 
                                    lr_quality_ethcat2 + 
                                    lr_quality_ethcat4 + 
                                    lr_quality_ethcat5 + 
                                    lr_quality_ethcat6 + 
                                    lr_quality_ethcat7
                                ),
        'hr_quality_total': np.mean(
                                    hr_quality_ethcat1 + 
                                    hr_quality_ethcat2 + 
                                    hr_quality_ethcat4 + 
                                    hr_quality_ethcat5 + 
                                    hr_quality_ethcat6 + 
                                    hr_quality_ethcat7
                                ),
        'e_quality_total': np.mean(
                                    e_quality_ethcat1 + 
                                    e_quality_ethcat2 + 
                                    e_quality_ethcat4 + 
                                    e_quality_ethcat5 + 
                                    e_quality_ethcat6 + 
                                    e_quality_ethcat7
                                ),
        'Waiting_time_mean': pd.concat([
                                    df_ethnicity1['waiting time (months)'],
                                    df_ethnicity2['waiting time (months)'],
                                    df_ethnicity4['waiting time (months)'],
                                    df_ethnicity5['waiting time (months)'],
                                    df_ethnicity6['waiting time (months)'],
                                    df_ethnicity7['waiting time (months)']
                                    ]).mean()


        }

In [45]:
def objective_function(m1, m2, m4, m5):
    
    multipliers_ethnicity = {
    1: m1,
    2: m2,
    4: m4,
    5: m5,
    6: 1.0,
    7: 1.0
}



    simulations_results = []
    for i in range(100):
        #print(f"Runing simulation {i + 1} of 100")
        results = run_simulation(
            total_time=10*12, 
            arrival_rate=(990/(10*12)), 
            departure_rate=((990/(10*12))*0.29), 
            match_run=3,
            pairs=pairs, 
            compatibility=compatibility, 
            hla_lr = hla_lr,
            hla_hr=hla_hr,
            hla_eplet= hla_eplet,
            multipliers_ethnicity= multipliers_ethnicity
        )
    
        simulations_results.append({
            'simulation': i + 1,
            'cycles': results['historial_cycles'],
            'total_cycles': len(results['historial_cycles']),
            'total_departures': len(results['historial_departures']),
            'total_entries': results['entries_nodes'],
            'arrivals_by_ethcat': results['arrivals_by_ethcat'],
            'waiting_time_mean1':   results['waiting_time_mean1'] ,
            'waiting_time_mean2':   results['waiting_time_mean2'],
            'waiting_time_mean4':   results['waiting_time_mean4'],
            'waiting_time_mean5':   results['waiting_time_mean5'],
            'waiting_time_mean6':   results['waiting_time_mean6'],
            'waiting_time_mean7':   results['waiting_time_mean7'],
            'lr_quality_ethcat1': results['lr_quality_ethcat1'],
            'lr_quality_ethcat2': results['lr_quality_ethcat2'],
            'lr_quality_ethcat4': results['lr_quality_ethcat4'],
            'lr_quality_ethcat5': results['lr_quality_ethcat5'],
            'lr_quality_ethcat6': results['lr_quality_ethcat6'],
            'lr_quality_ethcat7': results['lr_quality_ethcat7'],
            'hr_quality_ethcat1': results['hr_quality_ethcat1'],
            'hr_quality_ethcat2': results['hr_quality_ethcat2'],
            'hr_quality_ethcat4': results['hr_quality_ethcat4'],
            'hr_quality_ethcat5': results['hr_quality_ethcat5'],
            'hr_quality_ethcat6': results['hr_quality_ethcat6'],
            'hr_quality_ethcat7': results['hr_quality_ethcat7'],
            'e_quality_ethcat1': results['e_quality_ethcat1'],
            'e_quality_ethcat2': results['e_quality_ethcat2'],
            'e_quality_ethcat4': results['e_quality_ethcat4'],
            'e_quality_ethcat5': results['e_quality_ethcat5'],
            'e_quality_ethcat6': results['e_quality_ethcat6'],
            'e_quality_ethcat7': results['e_quality_ethcat7'],
            'avg_ClassI_HLA_ethcat1': results['avg_ClassI_HLA_ethcat1'],
            'avg_ClassI_HLA_ethcat2': results['avg_ClassI_HLA_ethcat2'],
            'avg_ClassI_HLA_ethcat4': results['avg_ClassI_HLA_ethcat4'],
            'avg_ClassI_HLA_ethcat5': results['avg_ClassI_HLA_ethcat5'],
            'avg_ClassI_HLA_ethcat6': results['avg_ClassI_HLA_ethcat6'],
            'avg_ClassI_HLA_ethcat7': results['avg_ClassI_HLA_ethcat7'],
            'HLA_ClassI_total': results['HLA_ClassI_total'],

            'avg_DR_HLA_ethcat1': results['avg_DR_HLA_ethcat1'],
            'avg_DR_HLA_ethcat2': results['avg_DR_HLA_ethcat2'],
            'avg_DR_HLA_ethcat4': results['avg_DR_HLA_ethcat4'],
            'avg_DR_HLA_ethcat5': results['avg_DR_HLA_ethcat5'],
            'avg_DR_HLA_ethcat6': results['avg_DR_HLA_ethcat6'],
            'avg_DR_HLA_ethcat7': results['avg_DR_HLA_ethcat7'],
            'HLA_DR_total': results['HLA_DR_total'],
            

            'avg_DQ_HLA_ethcat1': results['avg_DQ_HLA_ethcat1'],
            'avg_DQ_HLA_ethcat2': results['avg_DQ_HLA_ethcat2'],
            'avg_DQ_HLA_ethcat4': results['avg_DQ_HLA_ethcat4'],
            'avg_DQ_HLA_ethcat5': results['avg_DQ_HLA_ethcat5'],
            'avg_DQ_HLA_ethcat6': results['avg_DQ_HLA_ethcat6'],
            'avg_DQ_HLA_ethcat7': results['avg_DQ_HLA_ethcat7'],
            'HLA_DQ_total': results['HLA_DQ_total'],

            'F_per_ethcat':   results['F_per_ethcat'],   
            'L_per_ethcat':   results['L_per_ethcat'],   
            'F_total': results['F_total'],
            'lr_quality_total': results['lr_quality_total'],
            'hr_quality_total': results['hr_quality_total'],
            'e_quality_total': results['e_quality_total'],
            'Waiting_time_mean': results['Waiting_time_mean']
        })
        
        
    df_results = pd.DataFrame(simulations_results)
    F_P = np.mean(df_results['F_total'])
    HLA_P = np.mean(df_results['e_quality_total'])
    results_ethnicity = []

    for index, row in df_results.iterrows():
        nodes = set(nodo for ciclo in row['cycles'] for nodo in ciclo)
        df_nodes = recipients[recipients['Nodo'].isin(nodes)]
        count_ethnicity = df_nodes['ETHCAT'].value_counts().to_dict()
        results_ethnicity .append({'simulation': row['simulation'], **count_ethnicity})

    df_results_ethnicity  = pd.DataFrame(results_ethnicity ).fillna(0)
    arrivals_by_ethcat = pd.DataFrame(df_results['arrivals_by_ethcat'].tolist())
    arrivals_by_ethcat_mean = arrivals_by_ethcat.mean()
    target_ethnicity = [1, 2, 4, 5]
    m = sum(arrivals_by_ethcat_mean[e] for e in target_ethnicity)
    transplant_by_ethcat_mean = df_results_ethnicity.mean()
    percentage_by_ethnicity = (transplant_by_ethcat_mean) / arrivals_by_ethcat_mean
    quality = df_results[
        ['e_quality_ethcat1', 'e_quality_ethcat2', 'e_quality_ethcat4', 'e_quality_ethcat5']
    ].mean()

    kpi_total = 0

    for e in target_ethnicity:
        s = arrivals_by_ethcat_mean[e]
        F_s = percentage_by_ethnicity[e]
        HLA_s = quality[f'e_quality_ethcat{e}']
        term = (s / m) * (abs(F_s - F_P) + (1/m)*abs(HLA_s - HLA_P))
        kpi_total += term
        print(f"For ethcat {e} : s = {s}, m = {m}, F_s = {F_s}, F_P = {F_P}, HLA_s = {HLA_s}, HLA_P = {HLA_P} ")

    

    return kpi_total



In [46]:
import optuna
import numpy as np

def objective(trial):
    
    m1 = trial.suggest_float('m1', 0.5, 1.0, step=0.01) 
    m2 = trial.suggest_float('m2', 0.9, 1.8, step=0.01)  
    m4 = trial.suggest_float('m4', 0.9, 1.8, step=0.01)  
    m5 = trial.suggest_float('m5', 1.0, 2, step=0.01)  

    kpi = objective_function(m1, m2, m4, m5)
    return kpi


study = optuna.create_study(direction='minimize')
study.enqueue_trial({'m1': 1, 'm2': 1, 'm4': 1, 'm5': 1})
study.optimize(objective, n_trials=100)


print("Best value of m1 (ethcat 1):", study.best_params['m1'])
print("Best value of m2 (ethcat 2):", study.best_params['m2'])
print("Best value of m4 (ethcat 4):", study.best_params['m4'])
print("Best value of m5 (ethcat 5):", study.best_params['m5'])
print("Minimum KPI reached:", study.best_value)


[I 2025-08-11 18:33:27,482] A new study created in memory with name: no-name-bcb2d159-c23c-4cd0-a9ca-3b19be980c44
[I 2025-08-11 18:34:44,501] Trial 0 finished with value: 0.03941295348732311 and parameters: {'m1': 1.0, 'm2': 1.0, 'm4': 1.0, 'm5': 1.0}. Best is trial 0 with value: 0.03941295348732311.


For ethcat 1 : s = 592.13, m = 965.35, F_s = 0.6802053603093915, F_P = 0.6487062641464899, HLA_s = 96.66020353818035, HLA_P = 95.67059174904007 
For ethcat 2 : s = 170.39, m = 965.35, F_s = 0.6341334585362991, F_P = 0.6487062641464899, HLA_s = 93.93100266789394, HLA_P = 95.67059174904007 
For ethcat 4 : s = 147.33, m = 965.35, F_s = 0.5841308626892011, F_P = 0.6487062641464899, HLA_s = 94.60764689332804, HLA_P = 95.67059174904007 
For ethcat 5 : s = 55.5, m = 965.35, F_s = 0.5381981981981983, F_P = 0.6487062641464899, HLA_s = 92.37798982427198, HLA_P = 95.67059174904007 


[I 2025-08-11 18:36:03,591] Trial 1 finished with value: 0.05016772669114298 and parameters: {'m1': 0.97, 'm2': 1.1600000000000001, 'm4': 0.93, 'm5': 1.78}. Best is trial 0 with value: 0.03941295348732311.


For ethcat 1 : s = 590.39, m = 962.5600000000001, F_s = 0.6606480462067448, F_P = 0.6491523711547414, HLA_s = 94.752669281331, HLA_P = 93.78182988640789 
For ethcat 2 : s = 169.76, m = 962.5600000000001, F_s = 0.7323279924599434, F_P = 0.6491523711547414, HLA_s = 92.39675769653351, HLA_P = 93.78182988640789 
For ethcat 4 : s = 147.08, m = 962.5600000000001, F_s = 0.4881017133532771, F_P = 0.6491523711547414, HLA_s = 92.48221897559952, HLA_P = 93.78182988640789 
For ethcat 5 : s = 55.33, m = 962.5600000000001, F_s = 0.694559913247786, F_P = 0.6491523711547414, HLA_s = 91.26684862039967, HLA_P = 93.78182988640789 


[I 2025-08-11 18:37:24,810] Trial 2 finished with value: 0.0427322083791261 and parameters: {'m1': 0.61, 'm2': 0.97, 'm4': 1.59, 'm5': 1.12}. Best is trial 0 with value: 0.03941295348732311.


...

For ethcat 1 : s = 592.59, m = 965.1800000000001, F_s = 0.6455221991596213, F_P = 0.6487015108611799, HLA_s = 95.15728731735325, HLA_P = 94.21816035897557 
For ethcat 2 : s = 170.12, m = 965.1800000000001, F_s = 0.6061015753585705, F_P = 0.6487015108611799, HLA_s = 92.89269187275076, HLA_P = 94.21816035897557 
For ethcat 4 : s = 147.11, m = 965.1800000000001, F_s = 0.6966895520358914, F_P = 0.6487015108611799, HLA_s = 93.2345195216973, HLA_P = 94.21816035897557 
For ethcat 5 : s = 55.36, m = 965.1800000000001, F_s = 0.6828034682080925, F_P = 0.6487015108611799, HLA_s = 91.62303893663251, HLA_P = 94.21816035897557 


[I 2025-08-11 20:47:10,737] Trial 98 finished with value: 0.036950918208260186 and parameters: {'m1': 0.96, 'm2': 1.03, 'm4': 1.27, 'm5': 1.58}. Best is trial 97 with value: 0.019879737205662353.


For ethcat 1 : s = 590.47, m = 962.07, F_s = 0.6187105187393093, F_P = 0.6477599566793522, HLA_s = 94.47602276400364, HLA_P = 93.53529742242782 
For ethcat 2 : s = 169.48, m = 962.07, F_s = 0.6922940759971679, F_P = 0.6477599566793522, HLA_s = 92.15605002158573, HLA_P = 93.53529742242782 
For ethcat 4 : s = 146.92, m = 962.07, F_s = 0.7010618023414104, F_P = 0.6477599566793522, HLA_s = 92.8726536578152, HLA_P = 93.53529742242782 
For ethcat 5 : s = 55.2, m = 962.07, F_s = 0.6831521739130435, F_P = 0.6477599566793522, HLA_s = 91.04970940455729, HLA_P = 93.53529742242782 


[I 2025-08-11 20:48:32,546] Trial 99 finished with value: 0.03953894164070605 and parameters: {'m1': 0.6, 'm2': 1.1300000000000001, 'm4': 1.02, 'm5': 1.6099999999999999}. Best is trial 97 with value: 0.019879737205662353.


For ethcat 1 : s = 592.25, m = 965.1299999999999, F_s = 0.619636977627691, F_P = 0.6507077677789921, HLA_s = 94.36138455063573, HLA_P = 93.46865561640148 
For ethcat 2 : s = 170.04, m = 965.1299999999999, F_s = 0.7352975770406963, F_P = 0.6507077677789921, HLA_s = 92.18401891812677, HLA_P = 93.46865561640148 
For ethcat 4 : s = 147.4, m = 965.1299999999999, F_s = 0.6585481682496607, F_P = 0.6507077677789921, HLA_s = 92.80561929978727, HLA_P = 93.46865561640148 
For ethcat 5 : s = 55.44, m = 965.1299999999999, F_s = 0.7086940836940837, F_P = 0.6507077677789921, HLA_s = 91.2222546741688, HLA_P = 93.46865561640148 
Best value of m1 (ethcat 1): 0.96
Best value of m2 (ethcat 2): 0.96
Best value of m4 (ethcat 4): 1.37
Best value of m5 (ethcat 5): 1.69
Minimum KPI reached: 0.019879737205662353


In [47]:
for trial in study.trials:
    print(f"{trial.number} {trial.params['m1']} {trial.params['m2']} {trial.params['m4']} {trial.params['m5']} {trial.value}")


0 1.0 1.0 1.0 1.0 0.03941295348732311
1 0.97 1.1600000000000001 0.93 1.78 0.05016772669114298
2 0.61 0.97 1.59 1.12 0.0427322083791261
3 0.56 0.9500000000000001 0.97 1.25 0.03474652516209947
4 0.77 1.35 1.63 1.2 0.045459454819142965
5 0.99 1.2 1.34 1.67 0.03452982203852039
6 0.62 1.1300000000000001 1.1300000000000001 1.72 0.0379879486742837
7 0.6799999999999999 1.55 1.24 1.85 0.04186908903857025
8 0.67 1.1400000000000001 1.71 1.7000000000000002 0.04323771714576876
9 0.53 1.52 1.27 1.15 0.05069943452646463
10 0.86 1.75 1.46 1.48 0.04097833649228535
11 0.8500000000000001 0.9400000000000001 1.4100000000000001 1.45 0.03558845617362863
12 0.52 1.3 1.1 1.34 0.05115962827184663
13 0.89 1.22 1.1 1.98 0.03909285639083959
14 0.76 1.05 1.49 1.58 0.03845829055860278
15 0.81 1.4300000000000002 1.25 1.32 0.03847684122938711
16 0.94 0.92 1.78 1.6099999999999999 0.03723736359846269
17 0.69 1.28 0.9 1.37 0.035525097805791314
18 0.58 1.06 1.35 1.24 0.04337412303756319
19 0.9199999999999999 1.66000000000

In [48]:
import pandas as pd

df_trials = pd.DataFrame({
    'Trial': [t.number for t in study.trials],
    'm1': [t.params['m1'] for t in study.trials],
    'm2': [t.params['m2'] for t in study.trials],
    'm4': [t.params['m4'] for t in study.trials],
    'm5': [t.params['m5'] for t in study.trials],
    'KPI': [t.value for t in study.trials]
})

df_trials

,Trial,m1,m2,m4,m5,KPI
0,0,1.00,1.00,1.00,1.00,0.039413
1,1,0.97,1.16,0.93,1.78,0.050168
2,2,0.61,0.97,1.59,1.12,0.042732
3,3,0.56,0.95,0.97,1.25,0.034747
4,4,0.77,1.35,1.63,1.20,0.045459
...,...,...,...,...,...,...
95,95,0.94,1.09,1.34,1.66,0.036080
96,96,0.90,0.92,1.47,1.63,0.038162
97,97,0.96,0.96,1.37,1.69,0.019880
98,98,0.96,1.03,1.27,1.58,0.036951


In [49]:
# Mejores combinaciones
df_trials.sort_values('KPI').head(5)


,Trial,m1,m2,m4,m5,KPI
97,97,0.96,0.96,1.37,1.69,0.019880
82,82,0.94,0.94,1.42,1.53,0.022482
83,83,0.93,0.94,1.33,1.49,0.032289
79,79,0.91,1.03,1.27,1.62,0.033439
51,51,0.84,0.99,1.42,1.54,0.033977


**After of knowing m1, m2, m3 and m4**

In [58]:
#Insert the multipliers found by optuna

multipliers_ethnicity = {
    1: 0.96,
    2: 1.01,
    4: 1.06,
    5: 1.09,
    6: 1.0,
    7: 1.0
}

simulations_results = []
for i in range(100):
    print(f"Runing simulation {i + 1} of 100")
    results = run_simulation(
        total_time=10*12, 
        arrival_rate=(990/(10*12)), 
        departure_rate=((990/(10*12))*0.29), 
        match_run=3,
        pairs=pairs, 
        compatibility=compatibility, 
        hla_lr = hla_lr,
        hla_hr=hla_hr,
        hla_eplet= hla_eplet,
        multipliers_ethnicity= multipliers_ethnicity
    )
   
    simulations_results.append({
        'simulacion': i + 1,
        'cycles': results['historial_cycles'],
        'total_cycles': len(results['historial_cycles']),
        'total_departures': len(results['historial_departures']),
        'total_entries': results['entries_nodes'],
        'arrivals_by_ethcat': results['arrivals_by_ethcat'],
        'waiting_time_mean1':   results['waiting_time_mean1'] ,
        'waiting_time_mean2':   results['waiting_time_mean2'],
        'waiting_time_mean4':   results['waiting_time_mean4'],
        'waiting_time_mean5':   results['waiting_time_mean5'],
        'waiting_time_mean6':   results['waiting_time_mean6'],
        'waiting_time_mean7':   results['waiting_time_mean7'],
        'lr_quality_ethcat1': results['lr_quality_ethcat1'],
        'lr_quality_ethcat2': results['lr_quality_ethcat2'],
        'lr_quality_ethcat4': results['lr_quality_ethcat4'],
        'lr_quality_ethcat5': results['lr_quality_ethcat5'],
        'lr_quality_ethcat6': results['lr_quality_ethcat6'],
        'lr_quality_ethcat7': results['lr_quality_ethcat7'],
        'hr_quality_ethcat1': results['hr_quality_ethcat1'],
        'hr_quality_ethcat2': results['hr_quality_ethcat2'],
        'hr_quality_ethcat4': results['hr_quality_ethcat4'],
        'hr_quality_ethcat5': results['hr_quality_ethcat5'],
        'hr_quality_ethcat6': results['hr_quality_ethcat6'],
        'hr_quality_ethcat7': results['hr_quality_ethcat7'],
        'e_quality_ethcat1': results['e_quality_ethcat1'],
        'e_quality_ethcat2': results['e_quality_ethcat2'],
        'e_quality_ethcat4': results['e_quality_ethcat4'],
        'e_quality_ethcat5': results['e_quality_ethcat5'],
        'e_quality_ethcat6': results['e_quality_ethcat6'],
        'e_quality_ethcat7': results['e_quality_ethcat7'],
        'avg_ClassI_HLA_ethcat1': results['avg_ClassI_HLA_ethcat1'],
        'avg_ClassI_HLA_ethcat2': results['avg_ClassI_HLA_ethcat2'],
        'avg_ClassI_HLA_ethcat4': results['avg_ClassI_HLA_ethcat4'],
        'avg_ClassI_HLA_ethcat5': results['avg_ClassI_HLA_ethcat5'],
        'avg_ClassI_HLA_ethcat6': results['avg_ClassI_HLA_ethcat6'],
        'avg_ClassI_HLA_ethcat7': results['avg_ClassI_HLA_ethcat7'],
        'HLA_ClassI_total': results['HLA_ClassI_total'],

        'avg_DR_HLA_ethcat1': results['avg_DR_HLA_ethcat1'],
        'avg_DR_HLA_ethcat2': results['avg_DR_HLA_ethcat2'],
        'avg_DR_HLA_ethcat4': results['avg_DR_HLA_ethcat4'],
        'avg_DR_HLA_ethcat5': results['avg_DR_HLA_ethcat5'],
        'avg_DR_HLA_ethcat6': results['avg_DR_HLA_ethcat6'],
        'avg_DR_HLA_ethcat7': results['avg_DR_HLA_ethcat7'],
        'HLA_DR_total': results['HLA_DR_total'],
        

        'avg_DQ_HLA_ethcat1': results['avg_DQ_HLA_ethcat1'],
        'avg_DQ_HLA_ethcat2': results['avg_DQ_HLA_ethcat2'],
        'avg_DQ_HLA_ethcat4': results['avg_DQ_HLA_ethcat4'],
        'avg_DQ_HLA_ethcat5': results['avg_DQ_HLA_ethcat5'],
        'avg_DQ_HLA_ethcat6': results['avg_DQ_HLA_ethcat6'],
        'avg_DQ_HLA_ethcat7': results['avg_DQ_HLA_ethcat7'],
        'HLA_DQ_total': results['HLA_DQ_total'],

        'F_per_ethcat':   results['F_per_ethcat'],   
        'L_per_ethcat':   results['L_per_ethcat'],   
        'F_total': results['F_total'],
        'lr_quality_total': results['lr_quality_total'],
        'hr_quality_total': results['hr_quality_total'],
        'e_quality_total': results['e_quality_total'],
        'Waiting_time_mean': results['Waiting_time_mean']
    })
    

Runing simulation 1 of 100
Runing simulation 2 of 100
Runing simulation 3 of 100
Runing simulation 4 of 100
Runing simulation 5 of 100
Runing simulation 6 of 100
Runing simulation 7 of 100
Runing simulation 8 of 100
Runing simulation 9 of 100
Runing simulation 10 of 100
Runing simulation 11 of 100
Runing simulation 12 of 100
Runing simulation 13 of 100
Runing simulation 14 of 100
Runing simulation 15 of 100
Runing simulation 16 of 100
Runing simulation 17 of 100
Runing simulation 18 of 100
Runing simulation 19 of 100
Runing simulation 20 of 100
Runing simulation 21 of 100
Runing simulation 22 of 100
Runing simulation 23 of 100
Runing simulation 24 of 100
Runing simulation 25 of 100
Runing simulation 26 of 100
Runing simulation 27 of 100
Runing simulation 28 of 100
Runing simulation 29 of 100
Runing simulation 30 of 100
Runing simulation 31 of 100
Runing simulation 32 of 100
Runing simulation 33 of 100
Runing simulation 34 of 100
Runing simulation 35 of 100
Runing simulation 36 of 100
R

In [59]:
df_results = pd.DataFrame(simulations_results)

list_HLA_ClassI_total = df_results['HLA_ClassI_total'].tolist()


list_HLA_DR_total = df_results['HLA_DR_total'].tolist()


list_HLA_DQ_total = df_results['HLA_DQ_total'].tolist()



lists_ClassI = {
    1: df_results['avg_ClassI_HLA_ethcat1'].dropna().tolist(),
    2: df_results['avg_ClassI_HLA_ethcat2'].dropna().tolist(),
    4: df_results['avg_ClassI_HLA_ethcat4'].dropna().tolist(),
    5: df_results['avg_ClassI_HLA_ethcat5'].dropna().tolist(),
    6: df_results['avg_ClassI_HLA_ethcat6'].dropna().tolist(),
    7: df_results['avg_ClassI_HLA_ethcat7'].dropna().tolist()
}

lists_DR = {
    1: df_results['avg_DR_HLA_ethcat1'].dropna().tolist(),
    2: df_results['avg_DR_HLA_ethcat2'].dropna().tolist(),
    4: df_results['avg_DR_HLA_ethcat4'].dropna().tolist(),
    5: df_results['avg_DR_HLA_ethcat5'].dropna().tolist(),
    6: df_results['avg_DR_HLA_ethcat6'].dropna().tolist(),
    7: df_results['avg_DR_HLA_ethcat7'].dropna().tolist()
}

lists_DQ = {
    1: df_results['avg_DQ_HLA_ethcat1'].dropna().tolist(),
    2: df_results['avg_DQ_HLA_ethcat2'].dropna().tolist(),
    4: df_results['avg_DQ_HLA_ethcat4'].dropna().tolist(),
    5: df_results['avg_DQ_HLA_ethcat5'].dropna().tolist(),
    6: df_results['avg_DQ_HLA_ethcat6'].dropna().tolist(),
    7: df_results['avg_DQ_HLA_ethcat7'].dropna().tolist()
}



In [60]:
import scipy.stats as st

ethnicity = [1, 2, 4, 5, 6, 7]
n_sim = len(df_results)

def media_ci(list):
    m = np.mean(list)
    s = np.std(list, ddof=1)
    low, high = st.t.interval(0.95, len(list)-1, loc=m, scale=s/np.sqrt(len(list)))
    return m, f"{m:.3f} [{low:.3f}; {high:.3f}]"

rows = []
# Reporting per ethnicity 

for e in ethnicity:
    arrivals_e = df_results['arrivals_by_ethcat'].apply(lambda d: d.get(e, 0)).sum() / n_sim
    transplants_e = df_results['cycles'].apply(
        lambda cycles: sum(1 for c in cycles for n in c if recipients.loc[n, 'ETHCAT'] == e)
    ).sum() / n_sim

    F_vals = df_results['F_per_ethcat'].apply(lambda d: d[e])
    _, txt_F = media_ci(F_vals)
    std_F = np.std(F_vals, ddof=1)

    lr_quality = df_results[f'lr_quality_ethcat{e}']
    _, txt_antigen = media_ci(lr_quality)

    hr_quality = df_results[f'hr_quality_ethcat{e}']
    _, txt_allele = media_ci(hr_quality)

    e_quality  = df_results[f'e_quality_ethcat{e}']
    _, txt_eplet = media_ci(e_quality)

    waiting = df_results[f'waiting_time_mean{e}'].mean()

    L_vals = df_results['L_per_ethcat'].apply(lambda d: d[e])
    _, txt_L = media_ci(L_vals)

    f_mean = np.mean(F_vals)
    l_mean = np.mean(L_vals)
    still_in_kep = round(1 - f_mean - l_mean, 3)


    list_classi = lists_ClassI[e]

    _, txt_classi = media_ci(list_classi)

    list_dr = lists_DR[e]
    _, txt_dr = media_ci(list_dr)

    list_dq = lists_DQ[e]
    _, txt_dq = media_ci(list_dq)


    rows.append({
        'Ethnicity(s)': e,
        'Arrivals': round(arrivals_e, 2),
        'Transplants': round(transplants_e, 2),
        'F(s) (Matched)': txt_F,
        'HLA(s) Antigen': txt_antigen,
        'HLA(s) Allele': txt_allele,
        'HLA(s) Eplets': txt_eplet,
        'Waiting Time': round(waiting, 3),
        'L(s) (Left Unmatched)': txt_L,
        '1-F(s)-L(s) (Still in KEP)': still_in_kep,
        'HLA ClassI': txt_classi,
        'HLA DR': txt_dr,
        'HLA DQ': txt_dq,

    })

# Reporting the totals

total_arrivals = sum(df_results['arrivals_by_ethcat'].apply(lambda d: sum(d.values()))) / n_sim
total_transplants = df_results['cycles'].apply(lambda cycles: sum(len(c) for c in cycles)).sum() / n_sim
F_total = total_transplants / total_arrivals


F_s_total, txt_F_total = media_ci(df_results['F_total'])

L_vals_flat = (df_results['total_departures'] / df_results['total_entries']).tolist()
L_s_total, ic_L_total = media_ci(L_vals_flat)
txt_L_total = ic_L_total

_, txt_total_lr_quality = media_ci(df_results['lr_quality_total'])
_,txt_total_hr_quality = media_ci(df_results['hr_quality_total'])
_,txt_total_e_quality= media_ci(df_results['e_quality_total'])

txt_waiting_time_mean = np.mean(df_results['Waiting_time_mean'])

F_vals_flat = [df_results['F_per_ethcat'].iloc[i][e] for i in range(n_sim) for e in ethnicity]
std_F_total = np.std(F_vals_flat, ddof=1)

f_total_mean = F_total  
total_outgoing = sum(
    df_results['L_per_ethcat'].apply(lambda d: sum(d.values()))
) / n_sim

l_total_mean = total_outgoing / total_arrivals
still_in_kep_total = round(1 - F_total - l_total_mean, 3)


_, txt_classi_total = media_ci(list_HLA_ClassI_total)

_, txt_dr_total = media_ci(list_HLA_DR_total )

_, txt_dq_total = media_ci(list_HLA_DQ_total )

rows.append({
    'Ethnicity(s)': 'Entire Population',
    'Arrivals': round(total_arrivals, 2),
    'Transplants': round(total_transplants, 2),
    'F(s) (Matched)': txt_F_total,
    'HLA(s) Antigen': txt_total_lr_quality,
    'HLA(s) Allele': txt_total_hr_quality,
    'HLA(s) Eplets': txt_total_e_quality,
    'Waiting Time': txt_waiting_time_mean,
    'L(s) (Left Unmatched)': txt_L_total,
    '1-F(s)-L(s) (Still in KEP)': f"{(1 - L_s_total - F_s_total):.3f}",
    'HLA ClassI': txt_classi_total,
    'HLA DR': txt_dr_total,
    'HLA DQ': txt_dq_total
})

df_final_table = pd.DataFrame(rows)
display(df_final_table)

,Ethnicity(s),Arrivals,Transplants,F(s) (Matched),HLA(s) Antigen,HLA(s) Allele,HLA(s) Eplets,Waiting Time,L(s) (Left Unmatched),1-F(s)-L(s) (Still in KEP),HLA ClassI,HLA DR,HLA DQ
0,1,588.69,364.39,0.619 [0.615; 0.623],4.348 [4.334; 4.361],3.137 [3.124; 3.150],94.484 [94.383; 94.585],3.642000,0.318 [0.314; 0.323],0.063,2.340 [2.329; 2.351],0.896 [0.891; 0.902],1.112 [1.106; 1.118]
1,2,168.86,117.49,0.696 [0.688; 0.703],3.706 [3.679; 3.733],2.216 [2.191; 2.240],92.077 [91.930; 92.223],3.280000,0.262 [0.255; 0.269],0.043,1.846 [1.823; 1.870],0.789 [0.778; 0.799],1.071 [1.061; 1.081]
2,4,146.39,102.90,0.703 [0.695; 0.710],3.954 [3.926; 3.982],2.415 [2.389; 2.440],92.827 [92.660; 92.995],3.388000,0.257 [0.250; 0.264],0.041,2.104 [2.083; 2.125],0.819 [0.809; 0.829],1.031 [1.019; 1.043]
3,5,55.00,37.48,0.681 [0.670; 0.692],3.827 [3.786; 3.868],1.781 [1.744; 1.818],90.954 [90.694; 91.215],3.237000,0.276 [0.265; 0.287],0.043,2.016 [1.983; 2.048],0.764 [0.746; 0.782],1.048 [1.029; 1.066]
4,6,7.86,5.53,0.704 [0.676; 0.731],4.284 [4.177; 4.392],2.835 [2.714; 2.956],91.052 [90.346; 91.758],2.676000,0.255 [0.228; 0.282],0.042,2.084 [1.993; 2.176],0.951 [0.900; 1.003],1.249 [1.195; 1.302]
5,7,5.93,3.22,0.543 [0.510; 0.576],3.395 [3.263; 3.528],1.924 [1.793; 2.056],92.145 [91.202; 93.087],4.529000,0.366 [0.331; 0.401],0.091,1.641 [1.536; 1.747],0.711 [0.645; 0.776],1.043 [0.971; 1.116]
6,Entire Population,972.73,631.01,0.649 [0.645; 0.652],4.128 [4.119; 4.137],2.758 [2.750; 2.767],93.514 [93.445; 93.583],3.507876,0.296 [0.293; 0.300],0.055,2.184 [2.176; 2.193],0.855 [0.852; 0.859],1.088 [1.084; 1.092]


In [54]:
df_final_table # este tiene la dision con P 

,Ethnicity(s),Arrivals,Transplants,F(s) (Matched),HLA(s) Antigen,HLA(s) Allele,HLA(s) Eplets,Waiting Time,L(s) (Left Unmatched),1-F(s)-L(s) (Still in KEP),HLA ClassI,HLA DR,HLA DQ
0,1,592.09,384.16,0.649 [0.645; 0.652],4.444 [4.429; 4.458],3.203 [3.188; 3.217],95.303 [95.199; 95.407],3.605000,0.295 [0.291; 0.298],0.057,2.394 [2.382; 2.405],0.916 [0.910; 0.921],1.134 [1.129; 1.139]
1,2,169.59,102.75,0.606 [0.600; 0.612],3.807 [3.783; 3.832],2.327 [2.303; 2.351],92.900 [92.730; 93.071],3.879000,0.327 [0.320; 0.335],0.067,1.899 [1.877; 1.921],0.814 [0.803; 0.826],1.094 [1.083; 1.106]
2,4,147.42,103.01,0.699 [0.692; 0.706],4.008 [3.980; 4.036],2.458 [2.430; 2.485],93.313 [93.125; 93.501],3.389000,0.256 [0.249; 0.264],0.045,2.133 [2.110; 2.155],0.826 [0.813; 0.839],1.049 [1.037; 1.061]
3,5,55.43,38.36,0.692 [0.683; 0.701],3.868 [3.825; 3.912],1.849 [1.811; 1.886],91.385 [91.137; 91.634],3.308000,0.258 [0.249; 0.268],0.05,2.058 [2.024; 2.091],0.756 [0.738; 0.775],1.054 [1.036; 1.073]
4,6,7.93,6.03,0.761 [0.735; 0.787],4.336 [4.223; 4.448],2.955 [2.836; 3.074],91.145 [90.482; 91.808],2.695000,0.211 [0.186; 0.236],0.028,2.249 [2.146; 2.353],0.921 [0.877; 0.966],1.166 [1.116; 1.215]
5,7,5.94,3.85,0.648 [0.620; 0.676],3.330 [3.179; 3.482],1.831 [1.715; 1.948],92.182 [91.306; 93.057],4.692000,0.290 [0.259; 0.322],0.062,1.715 [1.606; 1.823],0.665 [0.608; 0.721],0.951 [0.882; 1.019]
6,Entire Population,978.40,638.16,0.652 [0.649; 0.655],4.229 [4.219; 4.239],2.850 [2.839; 2.860],94.301 [94.222; 94.380],3.597385,0.292 [0.288; 0.295],0.056,2.246 [2.239; 2.254],0.874 [0.870; 0.879],1.108 [1.104; 1.113]
